In [1]:
from calcula_resultados import compute_indicators_from_files, load_balancetes_to_df
import pandas as pd

files = ["balancetes/balancete1.json"]
result = compute_indicators_from_files(files)


In [11]:
# ---- DataFrame por arquivo ----
df_balancete = pd.json_normalize(
    result["per_file"]).drop(columns=["file", "n_leaves"])


df_balancete

,indicators.receita_bruta,indicators.impostos_vendas,indicators.receita_liquida,indicators.custos,indicators.lucro_bruto,indicators.despesas,indicators.lucro_liquido,indicators.disponibilidade_caixa
0,684563.19,3160.22,681402.97,405696.63,275706.34,416239.67,-140533.33,130876.58


In [7]:
df = load_balancetes_to_df(folder="balancetes/nfeat")

In [8]:
df

,mes_ref,periodo_text,codigo,conta,identificador,nivel,saldo_atual_valor,saldo_atual_sinal,saldo_anterior_valor,saldo_anterior_sinal,debito_valor,debito_sinal,credito_valor,credito_sinal,parent_codigo,saldo_atual_signed,mes_ref_dt
0,2023-01,2023-01,1,ATIVO,1 - ATIVO,0,4.718268e+09,D,NaN,,NaN,,NaN,,None,4.718268e+09,2023-01-01
1,2023-01,2023-01,1.1,ATIVO CIRCULANTE,1.1 - ATIVO CIRCULANTE,1,3.260144e+08,D,NaN,,NaN,,NaN,,1,3.260144e+08,2023-01-01
2,2023-01,2023-01,1.1.01,DISPONIBILIDADES,1.1.01 - DISPONIBILIDADES,2,1.452113e+07,D,NaN,,NaN,,NaN,,1.1,1.452113e+07,2023-01-01
3,2023-01,2023-01,1.1.01.01,CAIXA,1.1.01.01 - CAIXA,3,1.117010e+06,D,NaN,,NaN,,NaN,,1.1.01,1.117010e+06,2023-01-01
4,2023-01,2023-01,1.1.01.01.00001,Caixa,1.1.01.01.00001 - Caixa,4,1.015463e+05,,41269.69,,74245.34,,13968.7,,1.1.01.01,1.015463e+05,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2023-12,2023-12,1.2.01,IMOBILIZADO,1.2.01 - IMOBILIZADO,2,2.840715e+06,D,NaN,,NaN,,NaN,,1.2,2.840715e+06,2023-12-01
236,2023-12,2023-12,1.2.01.01,BENS E DIREITOS EM USO,1.2.01.01 - BENS E DIREITOS EM USO,3,2.162941e+05,D,NaN,,NaN,,NaN,,1.2.01,2.162941e+05,2023-12-01
237,2023-12,2023-12,1.2.01.01.00001,Máquinas e Equipamentos,1.2.01.01.00001 - Máquinas e Equipamentos,4,1.802451e+04,,0.00,,18024.51,,0.0,,1.2.01.01,1.802451e+04,2023-12-01
238,2023-12,2023-12,1.2.01.02,(-) DEPRECIAÇÃO ACUMULADA,1.2.01.02 - (-) DEPRECIAÇÃO ACUMULADA,3,2.222400e+03,D,NaN,,NaN,,NaN,,1.2.01,2.222400e+03,2023-12-01


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   mes_ref               240 non-null    object        
 1   periodo_text          240 non-null    object        
 2   codigo                240 non-null    object        
 3   conta                 240 non-null    object        
 4   identificador         240 non-null    object        
 5   nivel                 240 non-null    int64         
 6   saldo_atual_valor     239 non-null    float64       
 7   saldo_atual_sinal     240 non-null    object        
 8   saldo_anterior_valor  84 non-null     float64       
 9   saldo_anterior_sinal  240 non-null    object        
 10  debito_valor          84 non-null     float64       
 11  debito_sinal          240 non-null    object        
 12  credito_valor         84 non-null     float64       
 13  credito_sinal       